In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from joblib import dump

In [3]:
X_train = pd.read_csv('../data/processed/X_train_smote.csv')
y_train = pd.read_csv('../data/processed/y_train_smote.csv')
X_test = pd.read_csv('../data/processed/X_test_smote.csv')

In [6]:
y_train.head()

,drafted
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [7]:
y_train.values.ravel()

array([0., 0., 0., ..., 1., 1., 1.])

### 2. Logistic Classification

In [4]:
from sklearn.linear_model import LogisticRegression

In [39]:
y_mode = y_train['drafted'].mode().values[0]
y_mode

0.0

In [40]:
y_shape = (len(X_train), 1)
y_base = np.full(y_shape, y_mode)

In [41]:
from sklearn.metrics import roc_auc_score

In [45]:
baseline_model = pd.DataFrame({'roc-auc score': roc_auc_score(y_train, y_base)}, index=['Baseline'])

In [46]:
baseline_model

,roc-auc score
Baseline,0.5
